In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt

SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]


df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df.index = pd.to_datetime(df.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]


In [8]:
def mean_prediction(df, rate=24):
    data = pd.DataFrame([])
    for sym in SYMBOLS:
        data[sym] = df[sym].rolling(rate).mean()
    data.index = df.index
    return data
        
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    return data

rand = random_prediction(df)
mu = mean_prediction(df)

def condition(x):
    if x>0:
        return 1
    elif x< 0:
        return -1
    else:
        return 0

for sym in mu.columns:
    mu[sym] = mu[sym].map( lambda x: condition(x))
    
# mu.reset_index(inplace=True, drop=True)

display(rand.shape, mu.shape)
clas = mu.iloc[-1:]



(74593, 28)

(74593, 28)

In [9]:
pred_last = rand.iloc[-120:]
pred = rand.iloc[-1:]

pred = abs(pred)




,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-06-24 01:00:00+00:00,0.625357,-0.691137,0.211617,-0.103239,0.185589,-0.564786,0.250426,-0.256582,-0.486436,-0.551506,...,0.295264,0.159474,-0.091618,0.494052,0.535072,0.769263,0.632720,0.570185,0.330120,0.494642
2022-06-24 02:00:00+00:00,0.100656,-0.399946,-0.177322,-0.273245,0.384994,-0.577785,0.654998,-0.699224,0.751113,-0.574948,...,-0.466857,-0.753690,-0.036159,-0.518822,0.605729,-0.510610,-0.121954,0.737927,0.017305,0.136402
2022-06-24 03:00:00+00:00,-0.596913,-0.228258,-0.530878,0.309981,0.685244,-0.045780,0.494923,-0.464805,0.328859,0.474944,...,-0.605949,-0.428601,-0.369283,0.615416,-0.745563,0.229394,-0.467154,-0.058099,-0.265060,-0.229448
2022-06-24 04:00:00+00:00,0.616636,0.313529,0.155918,0.535756,-0.028740,-0.153774,-0.659824,0.756266,-0.244934,0.201907,...,0.090198,0.391905,0.568881,0.552697,0.716827,0.466452,0.114371,-0.487886,-0.025944,0.492159
2022-06-24 05:00:00+00:00,-0.700609,0.185309,-0.563636,0.230073,-0.741910,-0.628501,-0.464397,-0.411735,0.152591,0.329799,...,-0.590937,0.709263,0.244117,0.597594,-0.718742,0.367779,-0.477610,0.160763,-0.710214,0.547884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 20:00:00+00:00,-0.208620,0.627932,0.186594,0.420577,-0.473787,-0.666573,-0.190103,0.604191,0.471517,0.344843,...,-0.405461,0.568286,0.012541,-0.401010,-0.241666,0.350036,-0.115027,-0.107936,-0.709432,-0.119714
2022-06-30 21:00:00+00:00,0.000638,-0.492945,-0.401271,0.679954,0.146475,-0.236452,0.747435,0.559441,-0.799538,-0.422834,...,-0.593420,0.790910,-0.548477,-0.648697,0.497047,0.369811,0.645208,0.490658,-0.512586,0.534454
2022-06-30 22:00:00+00:00,-0.301039,-0.327313,-0.408090,-0.759569,-0.042073,0.219460,-0.074366,-0.388222,-0.260713,-0.012933,...,0.203903,0.425591,0.502860,0.199075,-0.366206,0.536444,-0.551211,-0.229796,-0.251451,-0.292993
2022-06-30 23:00:00+00:00,0.243772,0.549396,-0.486044,-0.351106,0.672954,0.426657,0.275148,0.748312,-0.647383,0.526356,...,0.268164,0.257814,0.781875,0.316008,-0.745685,0.139961,0.625047,0.736707,0.620801,0.536937


In [43]:
def correlation_pairs(df, rate=1):

    data = pd.DataFrame(index=df.index)
    for pair in df.columns:
        for second_pair in df.columns:
            data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
            # for the rolling correlation if we manage to do it...
            # data[f'{pair}_{second_pair}_corr'] = df[pair].rolling(rate).corr(df[second_pair]) 
    data.index = df.index
    data = data.iloc[-1:].T
    data.columns = ["correlation"]
    data = data[round(data["correlation"],6) != 1.000000]
    data = data.T
    
    return data

def weighted_volatility(volatility):
    weight = pd.DataFrame([])
    volatility = volatility.copy()
    volatility_T = volatility.T
    weight = volatility_T/ volatility_T.mean()
    return  weight

In [93]:
c = correlation_pairs(pred_last)

/tmp/ipykernel_19120/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_19120/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/tmp/ipykernel_19120/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [94]:
display(c, pred_last.iloc[-1:])

,AUD_CAD_AUD_CHF_corr,AUD_CAD_AUD_JPY_corr,AUD_CAD_AUD_NZD_corr,AUD_CAD_AUD_USD_corr,AUD_CAD_CAD_CHF_corr,AUD_CAD_CAD_JPY_corr,AUD_CAD_CHF_JPY_corr,AUD_CAD_EUR_AUD_corr,AUD_CAD_EUR_CAD_corr,AUD_CAD_EUR_CHF_corr,...,USD_JPY_GBP_CHF_corr,USD_JPY_GBP_JPY_corr,USD_JPY_GBP_NZD_corr,USD_JPY_GBP_USD_corr,USD_JPY_NZD_CAD_corr,USD_JPY_NZD_CHF_corr,USD_JPY_NZD_JPY_corr,USD_JPY_NZD_USD_corr,USD_JPY_USD_CAD_corr,USD_JPY_USD_CHF_corr
correlation,-0.18162,-0.029748,0.179353,0.044328,0.079784,0.053265,-0.018551,0.003121,-0.117923,0.058321,...,-0.218018,-0.139474,-0.037516,-0.100162,-0.013029,0.110798,-0.171514,0.012434,-0.027864,-0.130339


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0.016769,-0.583101,0.218011,0.713663,-0.503879,-0.197544,-0.206496,0.786118,-0.098419,-0.156443,...,0.46858,-0.427052,0.628566,-0.301348,-0.393826,-0.580397,0.025498,-0.041641,0.2301,0.612286


In [46]:
sym = 'AUD_JPY'
sym2 = 'AUD_CAD'

c.filter(regex=sym+'_'+sym2).values[0][0]

-0.029747597712141995

In [77]:
def expected_volatility(pred_vol, correlations):
    
    vols = pred_vol.copy()
    pred_vol = abs(pred_vol.iloc[:,-1:].T)
    for sym in pred_vol.columns:
        
        real_vol = 0
        
        for sym2 in pred_vol.columns:
            
            if sym == sym2:
                real_vol += pred_vol[sym]
            else:
                real_vol += pred_vol[sym2] * correlations.filter(regex=sym+'_'+sym2).values[0][0]
                
        vols[sym] = round(real_vol.mean(), 4)
        
    total_vol = vols.mean(axis=1)[0]

    return total_vol, vols
    
        


In [82]:
t, v = expected_volatility(pred,c)


print('Total volatilty:', t, '\nReal vols:\n', v)

Total volatilty: 0.38138356951194036 
Real vols:
                             AUD_CAD   AUD_CHF   AUD_JPY   AUD_NZD   AUD_USD  \
2022-07-01 00:00:00+00:00  0.016769  0.583101  0.218011  0.713663  0.503879   

                            CAD_CHF   CAD_JPY   CHF_JPY   EUR_AUD   EUR_CAD  \
2022-07-01 00:00:00+00:00  0.197544  0.206496  0.786118  0.098419  0.156443   

                           ...   GBP_NZD   GBP_USD   NZD_CAD   NZD_CHF  \
2022-07-01 00:00:00+00:00  ...  0.427052  0.628566  0.301348  0.393826   

                            NZD_JPY   NZD_USD   USD_CAD  USD_CHF   USD_JPY  \
2022-07-01 00:00:00+00:00  0.580397  0.025498  0.041641   0.2301  0.612286   

                           2022-07-01 00:00:00+00:00  
2022-07-01 00:00:00+00:00                     0.6123  

[1 rows x 29 columns]


In [55]:
clas["AUD_CAD"] = 0
clas

/tmp/ipykernel_19120/3202523835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clas["AUD_CAD"] = 0


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,0,1,-1,-1,1,1,-1,-1,-1,1,...,-1,-1,1,1,1,-1,1,-1,1,-1


In [97]:
def balance(pred_vols, classification, risk=1, balance=100000, leverage=1):
    cor = correlation_pairs(pred_vols)
    
    t,v = expected_volatility(pred_vols, cor)
    n = (abs(classification.T)).sum()
    risked = risk*balance*leverage/(t*n)
    print(risked)
    l = []
    for ccy in classification.columns:
        d = {}
        if int(classification[ccy]) != 0:
            d["ccy"] = ccy
            d["side"] = "sell" if int(classification[ccy]) == -1 else "buy"
            d["size"] = float(v[ccy].astype("float")*risked)
            d["datetime"] = pred_vols.index.to_list()[0].strftime("%Y-%m-%dT%H:%M:%S.Z")
            l.append(d)
        
    return l

In [98]:
ex = balance(pred, clas)
ex

/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/martillanes/.pyenv/versions/lewagon/lib/

2022-07-01 00:00:00+00:00    9711.230372
dtype: float64


/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/tmp/ipykernel_19120/3687875360.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df[pair].corr(df[second_pair])
/home/martillanes/.pyenv/versions/lewagon/lib/python3.8/site-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/martillanes/.pyenv/versions/lewagon/l

[{'ccy': 'AUD_CHF',
  'side': 'buy',
  'size': 5662.630138194666,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'AUD_JPY',
  'side': 'sell',
  'size': 2117.15339893029,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'AUD_NZD',
  'side': 'sell',
  'size': 6930.54650022202,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'AUD_USD',
  'side': 'buy',
  'size': 4893.280727387507,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'CAD_CHF',
  'side': 'buy',
  'size': 1918.398214210981,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'CAD_JPY',
  'side': 'sell',
  'size': 2005.328916839304,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'CHF_JPY',
  'side': 'sell',
  'size': 7634.172627263625,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'EUR_AUD',
  'side': 'sell',
  'size': 955.7731908796696,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'EUR_CAD',
  'side': 'buy',
  'size': 1519.2548073725516,
  'datetime': '2022-07-01T00:00:00.Z'},
 {'ccy': 'EUR_CHF',
  'side': 'buy',
  'si